Wednesday, August 23, 2017

> # Data preprocessing

In [424]:
import numpy as np
import pandas as pd
import nibabel as nb
import matplotlib.pyplot as plt
import re
import os
from xml.dom import minidom
from nipy.core.api import Image, vox2mni, rollimg, xyz_affine, as_xyz_image

pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [ ]:
a

## Current information csv file preprocessing

In [593]:
def get_layer_name_dict(fsl_xml):
    xmldoc = minidom.parse(fsl_xml)
    itemlist = xmldoc.getElementsByTagName('label')

    layer_name_dict = {}
    for s in itemlist:
        layer_name_dict[int(s.attributes['index'].value) + 1] = s.childNodes[0].data
    
    return layer_name_dict

In [641]:
def get_HO_region(img, x, y, z, layer_name_dict):    
    '''
    img : nibabel image class
    x,y,z in MNI mm coordinate
    layer_name_dict from fsl_xml
    '''
    # mm coordinate to voxel location
    voxel_loc = nb.affines.apply_affine(np.linalg.inv(img.affine), 
                                        [x,y,z]).round().astype('int')
    img_data = img.get_data()
    # layer number
    try:
        layer_number = img_data[voxel_loc[0], voxel_loc[1], voxel_loc[2]]
    except IndexError:
        layer_number = -1

    try:
        region = layer_name_dict[layer_number]
    except:
        region = 'error'
        
    return region, layer_number

In [566]:
HO_cortex_file_loc = '/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr0-1mm.nii.gz'
HO_cortex = nb.load(HO_cortex_file_loc)
HO_cortex_data = HO_cortex.get_data()

In [588]:
layer_xml = '/usr/local/fsl/data/atlases/HarvardOxford-Cortical.xml'
layer_dict = get_layer_name_dict(layer_xml)
layer_dict[0] = 'outside'

In [642]:
def current_file_preprocessing(csv):
    '''
    Returns 
    - strength df with matching brain region, 
    - 'normal' value df
    from the csv file containing the current information.
    
    The csv file has 5 lines of header : (not returned)
    - Latency
    - Residual Deviation (normalized)
    - Residual Deviation (original)
    - Explained Variance (normalized)
    - Explained Variance (original)
    
    Main dataframe is composed of three parts :
    1. Strength
        - for each activations along the latency
    2. Location
        - for each activations
    3. Normal
        - for each activations along the latency
        
    During the preprocessing, 
    - the location information is merged to the strength df, 
      based on the activation number.
    - Then the coordinate information is used 
      to find matching regions in the MNI space, 
      adding extra column of brain location the strength df.    
    '''
    
    # ISO-8859-1 encoding solves the encoding problem
    df_raw = pd.read_csv(csv, encoding='ISO-8859-1')
    df_header = df_raw.ix[:4]
    df_data = df_raw.ix[4:]
    
    # Extract information categories from the first column
    df_data['info'] = df_data['Latency [ms]'].str.split(' ').str[0]
    df_info_gb = df_data.groupby('info')
    
    # Strength df
    df_strength = df_info_gb.get_group('Strength')
    df_strength['number'] = df_strength['Latency [ms]'].str.split(' ').str[1].astype('int')
    df_strength = df_strength.drop('Latency [ms]', axis=1)
    
    # Location df
    df_location = df_info_gb.get_group('Location')
    df_location['number'] = df_location['Latency [ms]'].str.split(' ').str[1].astype('int')
    df_location['axis'] = df_location['Latency [ms]'].str.split(' ').str[2]
    # the csv has duplication of the coordinates for every latencies
    # select one
    first_num_col = df_location.columns[1]
    df_location = df_location[['info', 'number', 'axis', first_num_col]]
    df_location.columns = ['info', 'number', 'axis', 'coord']
    # x,y,z into columns
    df_location = df_location.pivot_table(index=['info','number'], 
                                          columns='axis', 
                                          values='coord')
    # get location using x,y,z mni coordinates
    df_location['mni_region'], df_location['mni_number'] = df_location.apply(
        lambda row: get_HO_region(
            HO_cortex, 
            row['x'], row['y'], row['z'], 
            layer_dict), 
        axis=1) 
    
    # merge strength with location
    df_strength_location = pd.merge(df_location.reset_index()[['number','x','y','z','mni_region']],
                                    df_strength,
                                    on='number', how='inner')
    
    # Normal df
    df_normal = df_info_gb.get_group('Normal')
    df_normal['number'] = df_normal['Latency [ms]'].str.split(' ').str[1].astype('int')
    df_normal['axis'] = df_normal['Latency [ms]'].str.split(' ').str[2]
    df_normal = df_normal.pivot_table(index=['info', 'number'], 
                                      columns='axis', 
                                      values=[x for x in df_normal.columns if re.search('\d',x)])
    return df_strength_location, df_normal

In [607]:
csv_location = u'/Users/kangik/Dropbox/project/2017_08_23_ERP_machine_learning/data/101027_Ctrl_LSJ2_current.csv'
csv_location = u'/Users/kangik/Dropbox/project/2017_08_23_ERP_machine_learning/data/101027_HC_LSJ2_MMN_current.csv'
csv_location = u'/Users/kangik/Dropbox/project/2017_08_23_ERP_machine_learning/data/101027_HC_LSJ2_P3_current.csv'

In [643]:
a,b = current_file_preprocessing(csv_location)

ValueError: Shape of passed values is (9524, 2), indices imply (9524, 3)

In [623]:
a.groupby('mni_region').groups.keys()

dict_keys(['Supramarginal Gyrus, posterior division', 'Inferior Temporal Gyrus, temporooccipital part', 'outside', 'Paracingulate Gyrus', 'Superior Temporal Gyrus, anterior division', 'Superior Temporal Gyrus, posterior division', 'Planum Temporale', 'Lateral Occipital Cortex, inferior division', 'Middle Frontal Gyrus', 'Lingual Gyrus', 'Temporal Occipital Fusiform Cortex', 'Parietal Operculum Cortex', 'Lateral Occipital Cortex, superior division', 'Middle Temporal Gyrus, posterior division', 'Central Opercular Cortex', 'Frontal Medial Cortex', 'Inferior Frontal Gyrus, pars triangularis', 'Cingulate Gyrus, anterior division', 'Intracalcarine Cortex', 'Supramarginal Gyrus, anterior division', "Heschl's Gyrus (includes H1 and H2)", 'Inferior Frontal Gyrus, pars opercularis', 'Planum Polare', 'Angular Gyrus', 'Inferior Temporal Gyrus, posterior division', 'Postcentral Gyrus', 'Occipital Pole', 'Cuneal Cortex', 'Middle Temporal Gyrus, anterior division', 'Subcallosal Cortex', 'Juxtapositio

In [625]:
a.head()

axis,number,x,y,z,mni_region,250.0,251.0,252.0,253.0,254.0,...,493.0,494.0,495.0,496.0,497.0,498.0,499.0,500.0,,info
0,1,69.0,-18.4,4.4,"Superior Temporal Gyrus, posterior division",0.016585,0.017600,0.018829,0.020196,0.021633,...,0.017893,0.017358,0.016918,0.016614,0.016481,0.016544,0.016810,0.017270,,Strength
1,2,69.9,-22.7,3.3,"Superior Temporal Gyrus, posterior division",0.024249,0.025292,0.026546,0.027953,0.029456,...,0.022657,0.021419,0.020141,0.018868,0.017652,0.016552,0.015626,0.014929,,Strength
2,3,69.6,-21.1,7.7,"Superior Temporal Gyrus, posterior division",0.025989,0.026860,0.027826,0.028859,0.029931,...,0.020635,0.020165,0.019751,0.019418,0.019187,0.019072,0.019083,0.019220,,Strength
3,4,69.6,-24.7,9.8,"Superior Temporal Gyrus, posterior division",0.031114,0.032179,0.033284,0.034410,0.035534,...,0.021863,0.021246,0.020643,0.020079,0.019576,0.019156,0.018836,0.018627,,Strength
4,5,70.3,-24.4,6.2,"Superior Temporal Gyrus, posterior division",0.028598,0.029507,0.030545,0.031678,0.032872,...,0.023139,0.022140,0.021114,0.020094,0.019115,0.018217,0.017436,0.016804,,Strength


In [640]:
a.head()#
new_columns = ['number', 'mni_region'] + [x for x in a.columns if re.search('\d', x)]
a[new_columns].set_index(['number', 'mni_region']).stack().reset_index()#, 'x', 'y', 'z', 'mni_region'])

,number,mni_region,axis,0
0,1,"Superior Temporal Gyrus, posterior division",250.0,0.016585
1,1,"Superior Temporal Gyrus, posterior division",251.0,0.017600
2,1,"Superior Temporal Gyrus, posterior division",252.0,0.018829
3,1,"Superior Temporal Gyrus, posterior division",253.0,0.020196
4,1,"Superior Temporal Gyrus, posterior division",254.0,0.021633
5,1,"Superior Temporal Gyrus, posterior division",255.0,0.023082
6,1,"Superior Temporal Gyrus, posterior division",256.0,0.024496
7,1,"Superior Temporal Gyrus, posterior division",257.0,0.025838
8,1,"Superior Temporal Gyrus, posterior division",258.0,0.027085
9,1,"Superior Temporal Gyrus, posterior division",259.0,0.028220


In [638]:
new_columns

['number',
 'mni_region',
 ' 250.0',
 ' 251.0',
 ' 252.0',
 ' 253.0',
 ' 254.0',
 ' 255.0',
 ' 256.0',
 ' 257.0',
 ' 258.0',
 ' 259.0',
 ' 260.0',
 ' 261.0',
 ' 262.0',
 ' 263.0',
 ' 264.0',
 ' 265.0',
 ' 266.0',
 ' 267.0',
 ' 268.0',
 ' 269.0',
 ' 270.0',
 ' 271.0',
 ' 272.0',
 ' 273.0',
 ' 274.0',
 ' 275.0',
 ' 276.0',
 ' 277.0',
 ' 278.0',
 ' 279.0',
 ' 280.0',
 ' 281.0',
 ' 282.0',
 ' 283.0',
 ' 284.0',
 ' 285.0',
 ' 286.0',
 ' 287.0',
 ' 288.0',
 ' 289.0',
 ' 290.0',
 ' 291.0',
 ' 292.0',
 ' 293.0',
 ' 294.0',
 ' 295.0',
 ' 296.0',
 ' 297.0',
 ' 298.0',
 ' 299.0',
 ' 300.0',
 ' 301.0',
 ' 302.0',
 ' 303.0',
 ' 304.0',
 ' 305.0',
 ' 306.0',
 ' 307.0',
 ' 308.0',
 ' 309.0',
 ' 310.0',
 ' 311.0',
 ' 312.0',
 ' 313.0',
 ' 314.0',
 ' 315.0',
 ' 316.0',
 ' 317.0',
 ' 318.0',
 ' 319.0',
 ' 320.0',
 ' 321.0',
 ' 322.0',
 ' 323.0',
 ' 324.0',
 ' 325.0',
 ' 326.0',
 ' 327.0',
 ' 328.0',
 ' 329.0',
 ' 330.0',
 ' 331.0',
 ' 332.0',
 ' 333.0',
 ' 334.0',
 ' 335.0',
 ' 336.0',
 ' 337.0',
 ' 338

In [629]:
a.pivot_table(index=['number', 'x', 'y', 'z', 'mni_region'], columns=['latency'],
             values = [x for x in a.columns if re.search('\d', x)])

KeyError: 'latency'

In [610]:
b.head()

250.0                251.0                252.0                \
axis               x      y      z      x      y      z      x      y      z   
info   number                                                                  
Normal 1      -0.872 -0.473 -0.127 -0.836 -0.500 -0.227 -0.797 -0.519 -0.308   
       2      -0.757 -0.623 -0.195 -0.722 -0.636 -0.273 -0.686 -0.645 -0.338   
       3      -0.901 -0.432 -0.037 -0.890 -0.447 -0.090 -0.877 -0.460 -0.138   
       4      -0.829 -0.548 -0.114 -0.818 -0.554 -0.152 -0.808 -0.559 -0.187   
       5      -0.769 -0.621 -0.152 -0.748 -0.630 -0.210 -0.726 -0.637 -0.261   

               253.0  ...    497.0  498.0                499.0                \
axis               x  ...        z      x      y      z      x      y      z   
info   number         ...                                                      
Normal 1      -0.761  ...   -0.295  0.555  0.808 -0.200  0.565  0.818 -0.104   
       2      -0.652  ...   -0.743  0.388  0.622 -0.680  0.412  0.685 -0.600   
       3      -0.863  ...   -0.438  0.385  0.842 -0.377  0.415  0.855 -0.312   
       4      -0.797  ...   -0.622  0.077  0.811 -0.580  0.119  0.839 -0.532   
       5      -0.704  ...   -0.745  0.217  0.684 -0.696  0.243  0.732 -0.636   

               500.0                
axis               x      y      z  
info   number                       
Normal 1       0.570  0.821 -0.010  
       2       0.435  0.748 -0.502  
       3       0.442  0.862 -0.247  
       4       0.161  0.863 -0.479  
       5       0.270  0.779 -0.566  

[5 rows x 753 columns]

## Peak information file preprocessing

In [796]:
def peak_preprocessing(textfile):
    df = pd.read_csv(text_data_loc, #skipfooter=1,
                 sep='\t', 
                 skiprows=5, 
                 names=['channel', 'x', 'y', 'z', 'minmax', 'latency'],
                 encoding='ISO-8859-1')

    #MGFP1 has only minmax and latency
    df.loc[df['channel']=='MGFP1', 'minmax'] = df.loc[df['channel']=='MGFP1', 'x']
    df.loc[df['channel']=='MGFP1', 'latency'] = df.loc[df['channel']=='MGFP1', 'y']

    df_melt = pd.melt(df, 
                      id_vars='channel', 
                      var_name='data', 
                      value_name='value', 
                      value_vars=['minmax', 'latency']).set_index(['channel', 'data']).T
    return df_melt

In [797]:
text_data_loc = u'/Users/kangik/Dropbox/project/2017_08_23_ERP_machine_learning/data/101027_HC_LSJ2_MMN_peak.txt'
text_data_loc = u'/Users/kangik/Dropbox/project/2017_08_23_ERP_machine_learning/data/101027_HC_LSJ2_P3_peak.txt'

In [798]:
d = peak_preprocessing(text_data_loc)

In [803]:
d

channel,FP1-avg,FPZ-avg,FP2-avg,AF3-avg,AF4-avg,F7 - avg,F5 - avg,F3 - avg,F1 - avg,FZ - avg,...,POZ-avg,PO4-avg,PO6-avg,PO8-avg,AF7-avg,O1 - avg,OZ - avg,O2 - avg,AF8-avg,MGFP1
data,minmax,minmax,minmax,minmax,minmax,minmax,minmax,minmax,minmax,minmax,...,latency,latency,latency,latency,latency,latency,latency,latency,latency,latency
value,-7.049,-5.449,-4.88,-3.424,-2.628,-5.184,-2.372,0.171,1.042,2.104,...,278.0,277.0,451.0,453.0,500.0,385.0,451.0,447.0,484.0,315.0
